## Read me

This page is a Jupyter notebook.  It is a combination of a document and some live software which you can execute if you are running your own jupyter-notebook server.  If you are not running a Jupyter server you can still read the document and see the code examples - you just can't run them.


## MiFID II classification of trades using the RTS 2 Annex 3 taxonomy.

Governments would prefer to avoid another financial crisis like the one in 2008 and beleive that making the big players operate in a more open and transparent way will help avoid another crash.

Markets in Financial Instruments Directive II (MiFID II) is an EU law which has market transparency as its key objective.  The predecessor law, MiFID I, only looked at a part of what banking firms do.  MiFID II aims to cover most mainstream activity.

Governments rely on regulators to make sure that their laws are being followed.  For MiFID II the primary regulator is ESMA.  ESMA have produced a number of Regulatory Technical Standard (RTS) documents which aim to explain what banking firms must do to comply with the MiFID II law.

One of the RTS documents, RTS 2, explains how different kinds of trading activity can be identified.  Having a clear way to say what has been traded is an important part of making the markets more transparent.

Some kinds of trading activity is already pretty transparent, for example buying and selling shares in a public company.  Trades of this kind are mostly done using a public exchange, such as the New York or London stock exchanges.  The 'price' for a given stock is simply the amount of money paid in the most recent trade and this price is made public by the exchange so everyone can see what the latest price is.  It is pretty easy to identify what has been traded because each stock has a identifier, e.g. 'AAPL' identifies Apple Inc. shares.

Not all trades happen on public exchanges.  Many trades happen directly between two parties and these are known as over the counter (OTC) trades.  Each OTC trade can be fine tuned, for example setting payment dates and interest rates.  The fine tuning of OTC deals makes it hard to identify what has been traded, but this is where RTS 2 comes in. 

The easyest way to understand what RTS 2 is all about is to use it to classify some trades, and you can do just that below.

### A Python Implementaion of RTS 2 Annex 3

It would be nice if ESMA published a working software implememtation of the RTS rules along with some test data so people can see exactly how the rules are supposed to work, and how reports are supposed to look.  But ESMA don't do that.  Each participant must somehow get an implementation of the RTS rules, either by writing it themselves or buying an implementation.

One market participant implemented the RTS rules themselves and have now released part of that implementaion under an open source license, the BSD license, so anyone can see the implementaion and use it.  This document forms a part of that release.

Hopefully this software will encourage ESMA to produce reference implementaions of their rules in future.  They could even take this software as a starting point.

The software here is written in the Python programming language.  This was done because Python is ubiquitous, that is it can be used easily and immediately on most modern computers; everything from a Raspberry Pi to the largest of big data clusters.

### Running a really simple initial classification

The box below contains python code which runs the classification sofware.  If you are just viewing this page then you won't be able to run the code, but if you start the page using your own local Jupyter notebook server then the code will really run if you select the box below and press control+enter


In [1]:
# Import the RTS 2 module and the Python date & time tools module
import rts2_annex3
import datetime

# Create a simple Python object to represent a trade.
class SampleTrade(object):
    pass
sample_trade = SampleTrade()
sample_trade.asset_class_name = 'Foreign Exchange Derivatives'
sample_trade.sub_asset_class_name= 'Deliverable FX options (DO)'
sample_trade.underlying_currency_pair = ('GBP', 'USD')
sample_trade.from_date = datetime.date(2017, 8, 13)
sample_trade.to_date = datetime.date(2017, 10, 12)

# Now classify the trade
sample_classification = rts2_annex3.class_root.classification_for(sample_trade)

# Lastly, display the classificaton
sample_classification.classification_dict()

{'Asset class': 'Foreign Exchange Derivatives',
 'RTS2 version': 'Brussels, 14.7.2016 C(2016) 4301 final ANNEXES 1 to 4',
 'Segmentation criterion 1': "('GBP', 'USD')",
 'Segmentation criterion 1 description': 'underlying currency pair defined as combination of the two currencies underlying the derivative contract',
 'Segmentation criterion 2': 'Maturity bucket 2: 1 week to 3 months',
 'Segmentation criterion 2 description': 'time to maturity bucket of the swap defined as follows:',
 'Sub-asset class': 'Deliverable FX options (DO)'}

### Understanding a classification

If you were able to run code above you would have seen something like this displayed, though the order of the results might not be the same:

```python
{
 'RTS2 version': 'Brussels, 14.7.2016 C(2016) 4301 final ANNEXES 1 to 4',

 'Asset class': 'Foreign Exchange Derivatives',

'Sub-asset class': 'Deliverable FX options (DO)', 

'Segmentation criterion 1 description': 'underlying currency pair defined as combination of the two currencies underlying the derivative contract',
 'Segmentation criterion 1': "('GBP', 'USD')",

'Segmentation criterion 2 description': 'time to maturity bucket of the swap defined as follows:',
 'Segmentation criterion 2': 'Maturity bucket 2: 1 week to 3 months', 
}
```

To a large degree, the classification is simply repeating the information we gave our trade object in the code above, but information has been checked and other information added.

This implementation first confirms the identity of the RTS document the classification is based upon.  The RTS rules may change over time, so it is important to know which version of the RTS a particular classification is based upon.

Next we see the Asset class and Sub-asset class, which is repeating just what we said above.  When classifying a trade there are some things you just have to know.  There will be some help on how to choose Asset classes and Sun-asset classes below.

Then we see something we didn't include in our trade object.  The RTS 2 Annex 3 document defines a number of criteria for each kind of Sub-asset class.  The Sub-asset class in this case has two criteria, and the classification included the description, the exact text, from the RTS document to explain what the crieria mean.

The values for the criteria do come from the values on our object, but some involve calculation.  The currencly pair criterion, criterion 1, is simply the name of underlying_currency_pair value we provided.  Criterion 2 gets its value from date calculations which use the from and to dates we gave, the resulting value is a date bucket, bucket 2 in this case.

### RTS 2 Annex 3 defines a taxonomy.  A Tree.

To understand how the classification process works we need to look at what the RTS says.

The RTS 2 Annex 3 taxonomy is made up of Asset classes which get broken down into Sub-asset classes which are further broken down by combinations of criteria values.

Here is some code to list the names of the asset classes:

In [2]:
# The root of the taxonomy is rts2_annex3.class_root.  This code asks the
# root for the asset classes, and for each asset class gets the name.
[asset_class.name for asset_class in rts2_annex3.class_root.asset_classes]

['Bonds (all bond types except ETCs and ETNs)',
 'Structured Finance Products (SFPs)',
 'Securitised Derivatives',
 'Interest Rate Derivatives',
 'Equity Derivatives',
 'Commodity Derivatives',
 'Foreign Exchange Derivatives',
 'Credit Derivatives',
 'C10 derivatives',
 'Financial contracts for differences (CFDs)',
 'Emission allowances',
 'Emission allowance derivatives']

Again, just in case you can't run the code here is what the list of Asset class names looks like:

```python
['Bonds (all bond types except ETCs and ETNs)',
 'Structured Finance Products (SFPs)',
 'Securitised Derivatives',
 'Interest Rate Derivatives',
 'Equity Derivatives',
 'Commodity Derivatives',
 'Foreign Exchange Derivatives',
 'Credit Derivatives',
 'C10 derivatives',
 'Financial contracts for differences (CFDs)',
 'Emission allowances',
 'Emission allowance derivatives']
```
There are exactly the names of the Asset classes you'll see in the RTS document.

Each asset class is broken down into Sub-asset classes.  Here is some code to list the Sub-asset classes for one of the Asset classes named above:


In [3]:
fx_asset_class = rts2_annex3.class_root.asset_class_by_name('Foreign Exchange Derivatives')
[sub_asset_class.name for sub_asset_class in fx_asset_class.children]

['Non-deliverable forward (NDF)',
 'Deliverable forward (DF)',
 'Non-Deliverable FX options (NDO)',
 'Deliverable FX options (DO)',
 'Non-Deliverable FX swaps (NDS)',
 'Deliverable FX swaps (DS)',
 'FX futures',
 'Other Foreign Exchange Derivatives']

... and we can then ask one of these Foreign Exchange Derivatives Sub-asset classes for the description of its criteria:

In [4]:
fx_do_sub_asset_class = fx_asset_class.sub_asset_class_by_name('Deliverable FX options (DO)')
[criterion.description for criterion in fx_do_sub_asset_class.criteria]

['underlying currency pair defined as combination of the two currencies underlying the derivative contract',
 'time to maturity bucket of the swap defined as follows:']

In [5]:
import ipywidgets as widgets

In [6]:
asset_classes = rts2_annex3.class_root.asset_classes
asset_class_names = [an_asset_class.name for an_asset_class in asset_classes]
asset_class_dict = dict([
        (an_asset_class.name, an_asset_class) 
        for an_asset_class 
        in asset_classes])

In [7]:
asset_class_widget = widgets.Dropdown(
    options=asset_class_dict,
    # value=asset_class_dict[asset_class_names[0]],
    description='Asset Classes:',
    disabled=False,
)
asset_class_widget

In [8]:
#selected_asset_class = rts2_annex3.class_root.asset_class_by_name(asset_class_widget.value)
selected_asset_class = asset_class_widget.value
sub_asset_class_names = [
    sub_asset_class.name 
    for sub_asset_class 
    in selected_asset_class.sub_asset_classes]
sub_asset_class_dict = dict([
    (sub_asset_class.name, sub_asset_class) 
    for sub_asset_class 
    in selected_asset_class.sub_asset_classes])

In [9]:
sub_asset_class_widget = widgets.Dropdown(
    options=sub_asset_class_dict,
    # value=sub_asset_class_names[0],
    description='Sub-asset Classes:',
    disabled=False,
)
sub_asset_class_widget

In [10]:
criteria_vbox = widgets.VBox([])

In [11]:
from IPython.display import display

In [13]:
def criteria_widgets(sub_asset_class):
    # OK, in here I need to look up the criteria for the
    # sub-asset class and build the widgets in rows of HBox es
    return [widgets.Label(criterion.display(prefix=""))
            for criterion
            in sub_asset_class.criteria]
    
def asset_class_changed(change):
    if change['type'] == 'change' and change['name'] == 'value':
        selected_asset_class = change['new']
        sub_asset_class_names = [sub_asset_class.name for sub_asset_class in selected_asset_class.sub_asset_classes]
        sub_asset_class_dict = dict([
            (sub_asset_class.name, sub_asset_class) 
            for sub_asset_class 
            in selected_asset_class.sub_asset_classes])
        sub_asset_class_widget.options = sub_asset_class_dict
        
def sub_asset_class_changed(change):
    if change['type'] == 'change' and change['name'] == 'value':
        selected_sub_asset_class = change['new']
        criteria_vbox.children = criteria_widgets(selected_sub_asset_class)

asset_class_widget.observe(asset_class_changed)
sub_asset_class_widget.observe(sub_asset_class_changed)
display(asset_class_widget)
display(sub_asset_class_widget)
criteria_vbox.children = criteria_widgets(sub_asset_class_widget.value)
display(criteria_vbox)


In [ ]:
from ipywidgets import interact
@interact(x=dict(a=1,b=2,c=3))
def xx(x):
    return x

In [ ]:
from IPython.display import display
button = widgets.Button(description="Click Me!")
display(button)

def on_button_clicked(b):
    print("Button clicked.")

button.on_click(on_button_clicked)